In [0]:
import os 
import sys
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .appName("Window Functions")\
        .getOrCreate()

In [0]:
simpleData = (("James", "Sales", 3000), \
    ("Michael", "Sales", 4600),  \
    ("Robert", "Sales", 4100),   \
    ("Maria", "Finance", 3000),  \
    ("James", "Sales", 3000),    \
    ("Scott", "Finance", 3300),  \
    ("Jen", "Finance", 3900),    \
    ("Jeff", "Marketing", 3000), \
    ("Kumar", "Marketing", 2000),\
    ("Saif", "Sales", 4100), \
    ("Ramesh", "Finance", 4100) \
  )
 
columns= ["employee_name", "department", "salary"]
df = spark.createDataFrame(data = simpleData, schema = columns)
df.printSchema()

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)



### Window Ranking Functions

#### 1. row_number

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
window=Window.partitionBy("department").orderBy("salary")
df.withColumn("row_number",row_number().over(window)).show()

+-------------+----------+------+----------+
|employee_name|department|salary|row_number|
+-------------+----------+------+----------+
|        Maria|   Finance|  3000|         1|
|        Scott|   Finance|  3300|         2|
|          Jen|   Finance|  3900|         3|
|       Ramesh|   Finance|  4100|         4|
|        Kumar| Marketing|  2000|         1|
|         Jeff| Marketing|  3000|         2|
|        James|     Sales|  3000|         1|
|        James|     Sales|  3000|         2|
|       Robert|     Sales|  4100|         3|
|         Saif|     Sales|  4100|         4|
|      Michael|     Sales|  4600|         5|
+-------------+----------+------+----------+



#### 2.rank()

In [0]:
from pyspark.sql.functions import rank
df.withColumn("rank",rank().over(window)).show(truncate=False)

+-------------+----------+------+----+
|employee_name|department|salary|rank|
+-------------+----------+------+----+
|Maria        |Finance   |3000  |1   |
|Scott        |Finance   |3300  |2   |
|Jen          |Finance   |3900  |3   |
|Ramesh       |Finance   |4100  |4   |
|Kumar        |Marketing |2000  |1   |
|Jeff         |Marketing |3000  |2   |
|James        |Sales     |3000  |1   |
|James        |Sales     |3000  |1   |
|Robert       |Sales     |4100  |3   |
|Saif         |Sales     |4100  |3   |
|Michael      |Sales     |4600  |5   |
+-------------+----------+------+----+



#### 3.dense_rank()

In [0]:
from pyspark.sql.functions import dense_rank
df.withColumn("dense_rank",dense_rank().over(window)).show()

+-------------+----------+------+----------+
|employee_name|department|salary|dense_rank|
+-------------+----------+------+----------+
|        Maria|   Finance|  3000|         1|
|        Scott|   Finance|  3300|         2|
|          Jen|   Finance|  3900|         3|
|       Ramesh|   Finance|  4100|         4|
|        Kumar| Marketing|  2000|         1|
|         Jeff| Marketing|  3000|         2|
|        James|     Sales|  3000|         1|
|        James|     Sales|  3000|         1|
|       Robert|     Sales|  4100|         2|
|         Saif|     Sales|  4100|         2|
|      Michael|     Sales|  4600|         3|
+-------------+----------+------+----------+



#### 4.percent_rank()

In [0]:
from pyspark.sql.functions import percent_rank
df.withColumn("percent_rank",percent_rank().over(window)).show()

+-------------+----------+------+------------------+
|employee_name|department|salary|      percent_rank|
+-------------+----------+------+------------------+
|        Maria|   Finance|  3000|               0.0|
|        Scott|   Finance|  3300|0.3333333333333333|
|          Jen|   Finance|  3900|0.6666666666666666|
|       Ramesh|   Finance|  4100|               1.0|
|        Kumar| Marketing|  2000|               0.0|
|         Jeff| Marketing|  3000|               1.0|
|        James|     Sales|  3000|               0.0|
|        James|     Sales|  3000|               0.0|
|       Robert|     Sales|  4100|               0.5|
|         Saif|     Sales|  4100|               0.5|
|      Michael|     Sales|  4600|               1.0|
+-------------+----------+------+------------------+



#### 5.ntile()

In [0]:
from pyspark.sql.functions import ntile
df.withColumn("ntile",ntile(3).over(window)).show()

+-------------+----------+------+-----+
|employee_name|department|salary|ntile|
+-------------+----------+------+-----+
|        Maria|   Finance|  3000|    1|
|        Scott|   Finance|  3300|    1|
|          Jen|   Finance|  3900|    2|
|       Ramesh|   Finance|  4100|    3|
|        Kumar| Marketing|  2000|    1|
|         Jeff| Marketing|  3000|    2|
|        James|     Sales|  3000|    1|
|        James|     Sales|  3000|    1|
|       Robert|     Sales|  4100|    2|
|         Saif|     Sales|  4100|    2|
|      Michael|     Sales|  4600|    3|
+-------------+----------+------+-----+



### Window analytic functions

#### 1.cume_dist()

In [0]:
from pyspark.sql.functions import cume_dist
df.withColumn("cume_dist",cume_dist().over(window)).show()

+-------------+----------+------+---------+
|employee_name|department|salary|cume_dist|
+-------------+----------+------+---------+
|        Maria|   Finance|  3000|     0.25|
|        Scott|   Finance|  3300|      0.5|
|          Jen|   Finance|  3900|     0.75|
|       Ramesh|   Finance|  4100|      1.0|
|        Kumar| Marketing|  2000|      0.5|
|         Jeff| Marketing|  3000|      1.0|
|        James|     Sales|  3000|      0.4|
|        James|     Sales|  3000|      0.4|
|       Robert|     Sales|  4100|      0.8|
|         Saif|     Sales|  4100|      0.8|
|      Michael|     Sales|  4600|      1.0|
+-------------+----------+------+---------+



#### 2.lag()

In [0]:
from pyspark.sql.functions import lag
df.withColumn("lag",lag("salary",1).over(window)).show()

+-------------+----------+------+----+
|employee_name|department|salary| lag|
+-------------+----------+------+----+
|        Maria|   Finance|  3000|null|
|        Scott|   Finance|  3300|3000|
|          Jen|   Finance|  3900|3300|
|       Ramesh|   Finance|  4100|3900|
|        Kumar| Marketing|  2000|null|
|         Jeff| Marketing|  3000|2000|
|        James|     Sales|  3000|null|
|        James|     Sales|  3000|3000|
|       Robert|     Sales|  4100|3000|
|         Saif|     Sales|  4100|4100|
|      Michael|     Sales|  4600|4100|
+-------------+----------+------+----+



#### 3.lead()

In [0]:
from pyspark.sql.functions import lead
df.withColumn("lead",lead("salary",2).over(window)).show()

+-------------+----------+------+----+
|employee_name|department|salary|lead|
+-------------+----------+------+----+
|        Maria|   Finance|  3000|3900|
|        Scott|   Finance|  3300|4100|
|          Jen|   Finance|  3900|null|
|       Ramesh|   Finance|  4100|null|
|        Kumar| Marketing|  2000|null|
|         Jeff| Marketing|  3000|null|
|        James|     Sales|  3000|4100|
|        James|     Sales|  3000|4100|
|       Robert|     Sales|  4100|4600|
|         Saif|     Sales|  4100|null|
|      Michael|     Sales|  4600|null|
+-------------+----------+------+----+



### Window aggregate functions

In [0]:
from pyspark.sql.functions import row_number,col,avg,max,sum,min
window2=Window.partitionBy("department")
df.withColumn("row",row_number().over(window))\
    .withColumn("avg",avg(col("salary")).over(window2))\
        .withColumn("max",max(col("salary")).over(window2))\
            .withColumn("min",min(col("salary")).over(window2))\
                .withColumn("sum",sum(col("salary")).over(window2)).show(truncate=False)


+-------------+----------+------+---+------+----+----+-----+
|employee_name|department|salary|row|avg   |max |min |sum  |
+-------------+----------+------+---+------+----+----+-----+
|Maria        |Finance   |3000  |1  |3575.0|4100|3000|14300|
|Scott        |Finance   |3300  |2  |3575.0|4100|3000|14300|
|Jen          |Finance   |3900  |3  |3575.0|4100|3000|14300|
|Ramesh       |Finance   |4100  |4  |3575.0|4100|3000|14300|
|Kumar        |Marketing |2000  |1  |2500.0|3000|2000|5000 |
|Jeff         |Marketing |3000  |2  |2500.0|3000|2000|5000 |
|James        |Sales     |3000  |1  |3760.0|4600|3000|18800|
|James        |Sales     |3000  |2  |3760.0|4600|3000|18800|
|Robert       |Sales     |4100  |3  |3760.0|4600|3000|18800|
|Saif         |Sales     |4100  |4  |3760.0|4600|3000|18800|
|Michael      |Sales     |4600  |5  |3760.0|4600|3000|18800|
+-------------+----------+------+---+------+----+----+-----+

